# Chapter 7: Function Decorators and Closures

The goal of this chapter is to explain how function decorators work. This requires understanding:

* How Python evaluates decorator syntax

* How Python decides whether a variable is local

* Why closures exist and how the work

* What problem is solved by `nonlocal`

This is helpful when tackling further decorator topics, such as:

* Implementing a well-behaved decorator

* Interesting decorators in the standard library

* Implementing a parameterised decorator

## Decorators 101

A decorator is a callable that takes another function as an argument (the decorated function).

The decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.

In [1]:
# Example 7-1. A decorater usually replaces a function with a different one.
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print("running target()")
    
target()

running inner()


## When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined.

This is usually at *import time* (i.e. when a module is loaded by Python).

In [3]:
# Example 7-2. The registration.py module.
registery = []

def register(func):
    print("running register(%s)" % func)
    registery.append(func)
    return func

@register
def f1():
    print("running f1()")
    
@register
def f2():
    print("running f2()")
    
def f3():
    print("running f3()")
    
def main():
    print("running main()")
    print("registery ->", registery)
    f1()
    f2()
    f3()
    
if __name__ == "__main__":
    main()
    

running register(<function f1 at 0x1082d65e0>)
running register(<function f2 at 0x1082d68b0>)
running main()
registery -> [<function f1 at 0x1082d65e0>, <function f2 at 0x1082d68b0>]
running f1()
running f2()
running f3()


Note that `register` runs twice before any other function in the module. 

When `register` is called, it receives as an argument the function object being decorated.

After the module is loaded, the `registery` holds references to the two decorated functions.

These functions are only executed when explicitly called by `main`.

The above example is unusual in two ways:

1. The decorated function is defined in the same module as the decorated function. A real decorator is usually defined in one module and applied to functions in another.

2. The `register` decorator returns the same functions passed as an argument. In practice, most decorators define an inner function and return it.

If *registration.py* is imported (and not run as a script), the output is:

In [7]:
from examples import registration

running register(<function f1 at 0x10a6a21f0>)
running register(<function f2 at 0x10a7ec3a0>)


In [9]:
registration.registery

[<function examples.registration.f1()>, <function examples.registration.f2()>]

The main point here is to emphasize that function decorators are run as soon as the module is imported, but the decorated functions only run when explicitly invoked (*import time* vs. *runtime*).

## Decorator-Enhanced Strategy Pattern

Decorators can be used as a good enhancement to the [Strategy](https://en.wikipedia.org/wiki/Strategy_pattern) pattern presented in the previous chapter.

This solution has several advantages compared to those presented in the previous chapter (**p. 174**):

* Strategy functions don't need special names (i.e. drop `_promo` suffix).

* `@promotion` highlights the purpose of the decorated function and makes it easy to disable a promotion (just comment out the decorator).

* Discount strategies may be implemented in other modules, anywhere on the system, as long as the `@promotion` decorator is applied to them.

In [10]:
# Example 7-3. The promos list is filled by the promotion decorator
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points."""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item(order):
    """10% discount for line orders with 20 or more units."""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


@promotion
def large_order(order):
    "7% discount on orders with at least 10 distinct items."
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10 :
        return order.total() * .07
    else:
        return 0


def best_promo(order):
    """Select best discount available."""
    return max(promo(order) for promo in promos)

## Variable Scope Rules

Decorators almost always return inner functions.

Code the uses inner functions almost always depends on **closures** to operate correctly.

To understand closures, we need to look at how variable scopes work in Python.